In [1]:
import scanpy as sc
import anndata
import pandas as pd
from itertools import permutations
import os
from tqdm import tqdm

In [2]:
ann = sc.read('../../20250327_merge_4mosq/4.ann.h5ad')
# ann.obs_names = ann.obs_names.map(lambda x: x[:-3] if x.endswith('dv-dv') or x.endswith('bl-bl') else x)
ann

AnnData object with n_obs × n_vars = 42169 × 8161
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'mt_outlier', 'genes_outlier', 'batch', '_scvi_batch', '_scvi_labels', 'leiden_res', 'pre_name', 'cluster', 'cluster_name'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'batch_colors', 'leiden', 'leiden_res_colors', 'log1p', 'neighbors', 'umap'
    obsm: 'X_scVI', 'X_scVI_MDE', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [3]:
aae_path = '/public21/home/sc90258/huangying/analysis/12.pan_mosquito/20250318_replot_mosq/aae/1.QC.h5ad'
alb_path = '/public21/home/sc90258/huangying/analysis/12.pan_mosquito/20250318_replot_mosq/alb/1.QC.h5ad'
cxtri_path = '/public21/home/sc90258/huangying/analysis/12.pan_mosquito/20250318_replot_mosq/cxtri/1.QC.h5ad'
cxpip_path = '/public21/home/sc90258/huangying/analysis/12.pan_mosquito/20250318_replot_mosq/cxpip/1.QC.h5ad'

aae = sc.read(aae_path)
aae.obs_names = [f'{i}-aae' for i in aae.obs_names]
alb = sc.read(alb_path)
alb.obs_names = [f'{i}-alb' for i in alb.obs_names]
cxtri = sc.read(cxtri_path)
cxtri.obs_names = [f'{i}-cxtri' for i in cxtri.obs_names]
cxpip = sc.read(cxpip_path)
cxpip.obs_names = [f'{i}-cxpip' for i in cxpip.obs_names]

In [4]:
homog = pd.read_csv('/public21/home/sc90258/huangying/analysis/12.pan_mosquito/20240830_homo_gene/mosquito_all_homog.csv')
homog

,aae,alb,cxpip,cxtri
0,LOC110673977,LOC109409628,LOC120416324,Ctri05375
1,LOC110673980,LOC115258594,LOC120427986,Ctri05761
2,LOC110673981,LOC109427304,LOC120415083,Ctri12482
3,LOC110673983,LOC109402949,LOC120425061,Ctri03408
4,LOC110673984,LOC134285458,LOC120428736,Ctri02287
...,...,...,...,...
8167,LOC5580326,LOC109420085,LOC120419188,Ctri05472
8168,LOC5580327,LOC109420086,LOC120419195,Ctri05473
8169,LOC5580329,LOC109420083,LOC120419194,Ctri05470
8170,LOC5580330,LOC109420082,LOC120422788,Ctri05866


In [42]:
odir = '1.input'
for trt, ctl in tqdm(permutations(['aae', 'alb', 'cxpip', 'cxtri'], 2)):
    opath = os.path.join(odir, f'{trt}_vs_{ctl}')
    if not os.path.exists(opath):
        os.mkdir(opath)
    print(f'> Write files to dir: {opath}')

    adata = eval(ctl)
    dct = dict(homog[[ctl, trt]].values)
    var_names_sub = [i for i in adata.var_names if i in dct.keys()]
    homo_var_names = [dct[i] for i in adata.var_names if i in dct.keys()]
    adata_homo = adata[:, var_names_sub]
    adata_homo.var_names = homo_var_names

    eval(trt).obs_names_make_unique()
    adata_homo.obs_names_make_unique()
    eval(trt).var_names_make_unique()
    adata_homo.var_names_make_unique()

    adata_merge = anndata.concat(
        [eval(trt), adata_homo], join="inner", label="batch", keys=[trt, ctl])

    adata_merge.obs['cluster'] = ann.obs[
        ann.obs['batch'].isin([trt, ctl])]['cluster']
    adata_merge = adata_merge[~adata_merge.obs['cluster'].isna()]
    adata_merge.obs['cmp'] = adata_merge.obs['batch'].map(
        lambda x: 1 if x == trt else 0)

    for cell in tqdm(adata_merge.obs['cluster'].unique()):
        ofile = os.path.join(opath, cell.replace(r'/', '-') + '.h5ad')
        adata_merge[adata_merge.obs['cluster'] == cell].write(ofile)

0it [00:00, ?it/s]

> Write files to dir: 1.input/aae_vs_alb


/tmp/ipykernel_105702/1869771834.py:26: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_merge.obs['cmp'] = adata_merge.obs['batch'].map(
100%|██████████| 11/11 [00:01<00:00,  9.79it/s]
1it [00:02,  2.87s/it]

> Write files to dir: 1.input/aae_vs_cxpip


/tmp/ipykernel_105702/1869771834.py:26: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_merge.obs['cmp'] = adata_merge.obs['batch'].map(
100%|██████████| 11/11 [00:00<00:00, 13.14it/s]
2it [00:04,  2.33s/it]

...

/tmp/ipykernel_105702/1869771834.py:26: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_merge.obs['cmp'] = adata_merge.obs['batch'].map(
100%|██████████| 11/11 [00:01<00:00,  6.69it/s]
10it [00:28,  2.95s/it]

> Write files to dir: 1.input/cxtri_vs_alb


/tmp/ipykernel_105702/1869771834.py:26: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_merge.obs['cmp'] = adata_merge.obs['batch'].map(
100%|██████████| 11/11 [00:02<00:00,  4.57it/s]
11it [00:32,  3.25s/it]

> Write files to dir: 1.input/cxtri_vs_cxpip


/tmp/ipykernel_105702/1869771834.py:26: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_merge.obs['cmp'] = adata_merge.obs['batch'].map(
100%|██████████| 11/11 [00:02<00:00,  3.83it/s]
12it [00:36,  3.01s/it]


In [5]:
odir = '1.input'

def gname_to_aae(adata, sp):
    dct = dict(homog[[sp, 'aae']].values)
    var_names_sub = [i for i in adata.var_names if i in dct.keys()]
    homo_var_names = [dct[i] for i in adata.var_names if i in dct.keys()]
    adata_homo = adata[:, var_names_sub]
    adata_homo.var_names = homo_var_names

    return adata_homo

alb_homo = gname_to_aae(alb, 'alb')
cxpip_homog = gname_to_aae(cxpip, 'cxpip')
cxtri_homo = gname_to_aae(cxtri, 'cxtri')

adata_merge = anndata.concat(
    [aae, alb_homo, cxtri_homo, cxpip_homog], 
    join="inner", label="batch", keys=['aae', 'alb', 'cxtri', 'cxpip'])

adata_merge.obs['cluster'] = ann.obs['cluster']
adata_merge = adata_merge[~adata_merge.obs['cluster'].isna()]

for trt, ctl in permutations([['aae', 'alb'], ['cxtri', 'cxpip']], 2):

    opath = os.path.join(odir, f'{'.'.join(trt)}_vs_{'.'.join(ctl)}')
    if not os.path.exists(opath):
        os.mkdir(opath)
    print(f'> Write files to dir: {opath}')

    adata_merge.obs['cmp'] = adata_merge.obs['batch'].map(
        lambda x: 1 if x in trt else 0)

    for cell in tqdm(adata_merge.obs['cluster'].unique()):
        ofile = os.path.join(opath, cell.replace(r'/', '-') + '.h5ad')
        adata_merge[adata_merge.obs['cluster'] == cell].write(ofile)

> Write files to dir: 1.input/aae.alb_vs_cxtri.cxpip


/tmp/ipykernel_27021/2916162298.py:30: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_merge.obs['cmp'] = adata_merge.obs['batch'].map(
100%|██████████| 11/11 [00:01<00:00,  8.42it/s]


> Write files to dir: 1.input/cxtri.cxpip_vs_aae.alb


100%|██████████| 11/11 [00:01<00:00,  9.51it/s]
